In [1]:
import numpy as np
import pandas as pd

In [34]:
def _read_observation_file(filename):
    df = pd.read_csv(filename, sep=r" \s+", engine="python")
    return df


def write_data(df, filename_out):
    """writes the data extracted from input and result files to
    filename_out

    Parameters
    ----------
    df : pd.DataFrame
        data to be written
    filename_out : string
        name of the output file in which the data should be written
    """
    header = "   ".join(df.columns) + "\n"

    with open(filename_out, "w") as f:
        f.write(header)

    # convert the list of dicts to a dataframe
    for col in df.columns[1:]:
        df[col] = df[col].apply(lambda x: f"{float(x):.4E}")

    # fuse the columns of the dataframe
    df_1col = df.apply(lambda row: "   ".join(list(row)) + "\n", 1)

    # save to output file
    with open(filename_out, "a") as f:
        for i in range(len(df_1col)):
            f.write(df_1col.at[i])

In [39]:
with_cp = True

mode = "err"
df = _read_observation_file(f"./omc1-{mode}-cgs.dat")

df["X"] = df["X_Y"].apply(lambda x: x.split("_")[0])
df["Y"] = df["X_Y"].apply(lambda x: x.split("_")[1])
df = df.drop(["X_Y"], 1)
df = df.astype(float)
df[["X", "Y"]] = df[["X", "Y"]].astype(int)
df = df.set_index(["X", "Y"])
df = df.sort_index()

df.head()

/tmp/ipykernel_372367/2867555930.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(["X_Y"], 1)


13CO J=2-1  C+ 158um     CH+ J=1-0  CO J=10-9      CO J=2-1  HCO+ J=3-2
X Y                                                                           
0 0  3.031814e-09       NaN           NaN   0.000002  1.751060e-09         NaN
  1  2.692184e-09  0.000019  1.454706e-07   0.000002  1.884334e-09         NaN
  2  2.618979e-09  0.000017  1.227631e-07   0.000002  1.954103e-09         NaN
  3  2.505580e-09  0.000015  1.284205e-07   0.000002  1.948343e-09         NaN
  4  2.252483e-09  0.000011  1.148440e-07   0.000002  2.204599e-09         NaN

In [40]:
df = df.rename(
    columns={
        "CH+ J=1-0": "chp_j1__j0",
        "CO J=10-9": "co_v0_j10__v0_j9",
        "CO J=2-1": "co_v0_j2__v0_j1",
        "13CO J=2-1": "13c_o_j2__j1",
        "HCO+ J=3-2": "hcop_j3__j2",
        "C+ 158um": "cp_el2p_j3_2__el2p_j1_2",
    }
)

In [41]:
df.columns

Index(['13c_o_j2__j1', 'cp_el2p_j3_2__el2p_j1_2', 'chp_j1__j0',
       'co_v0_j10__v0_j9', 'co_v0_j2__v0_j1', 'hcop_j3__j2'],
      dtype='object')

In [42]:
df.to_pickle(f"./omc1-{mode}-cgs.pkl")

In [ ]:
df

In [33]:
df.head()

13co_v0_j2__v0_j1  cp_el2p_j3_2__el2p_j1_2    chp_j1__j0  \
X Y                                                             
0 0       6.434450e-08                      NaN           NaN   
  1       6.611934e-08                 0.001511  2.809610e-07   
  2       7.151950e-08                 0.001501  3.581804e-07   
  3       8.042203e-08                 0.001491  9.968447e-07   
  4       9.104496e-08                 0.001465  3.233371e-07   

     co_v0_j10__v0_j9  co_v0_j2__v0_j1  hcop_j3__j2  
X Y                                                  
0 0         -0.000002     9.762374e-07          NaN  
  1         -0.000002     1.005557e-06          NaN  
  2         -0.000002     1.087570e-06          NaN  
  3         -0.000002     1.223851e-06          NaN  
  4         -0.000004     1.345087e-06          NaN

In [19]:
# with_cp_str = "with_Cp" if with_cp else "without_Cp"

# write_data(df, f"./omc1-{mode}-{with_cp_str}-cgs_fit.dat")

In [32]:
df.groupby("idx")["idx"].count().sort_values()

KeyError: 'idx'

In [21]:
df_test = pd.read_pickle(f"./omc1-{mode}-cgs.pkl")

In [22]:
df_test

chp_j1__j0  co_v0_j10__v0_j9  co_v0_j2__v0_j1   hcop_j3__j2  \
X  Y                                                                   
0  0            NaN          0.000002     1.751100e-09           NaN   
   1   1.454700e-07          0.000002     1.884300e-09           NaN   
   2   1.227600e-07          0.000002     1.954100e-09           NaN   
   3   1.284200e-07          0.000002     1.948300e-09           NaN   
   4   1.148400e-07          0.000002     2.204600e-09           NaN   
...             ...               ...              ...           ...   
40 56  1.053300e-07          0.000001     3.332300e-09  5.727500e-08   
   57  1.669700e-07          0.000001     2.997400e-09  5.193800e-08   
   58  1.787800e-07          0.000002     2.908000e-09  4.216200e-08   
   59           NaN          0.000001     2.308000e-09  5.423900e-08   
   60           NaN          0.000002     3.048300e-09  7.218700e-08   

       cp_el2p_j3_2__el2p_j1_2  
X  Y                            
0  0                       NaN  
   1                  0.000019  
   2                  0.000017  
   3                  0.000015  
   4                  0.000011  
...                        ...  
40 56                 0.000012  
   57                 0.000015  
   58                 0.000017  
   59                 0.000018  
   60                      NaN  

[2476 rows x 5 columns]